<a href="https://colab.research.google.com/github/kidrono/Kaggle-competitions/blob/main/kaggle_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle competition: Natural Language Processing with Disaster Tweets
The competition information could be found [here](https://www.kaggle.com/competitions/nlp-getting-started/overview)

In [82]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [84]:
! cp kaggle.json ~/.kaggle/

In [85]:
! chmod 600 ~/.kaggle/kaggle.json

In [86]:
! kaggle competitions download nlp-getting-started

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [87]:
import zipfile
import os

if os.path.isfile("/data/train.csv"):
  print("allready exists")
else:
  zip_ref = zipfile.ZipFile("nlp-getting-started.zip", 'r')
  zip_ref.extractall("/data")
  zip_ref.close()

allready exists


In [88]:
import pandas as pd
import numpy as np
import re
import torch

In [89]:
train_df = pd.read_csv("/data/train.csv")
train_labels = train_df['target']

In [90]:
# one_hot_dict = dict()
# i = 0
# for text in train_df['text']:
#   clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
#   tweet = clean_tweet.split(' ')
#   for word in tweet:
#     word = word.lower()
#     if word not in one_hot_dict.keys():
#       one_hot_dict[word] = i
#       i += 1
# vec_dim = len(one_hot_dict) + 1
# vec_dim

In [91]:
# def sentence_vector(text):
#   vec = np.zeros(vec_dim)
#   clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
#   tweet = clean_tweet.split(' ')
#   for word in tweet:
#     word = word.lower()
#     if word in one_hot_dict.keys():
#       vec[one_hot_dict[word]] += 1
#     else:
#       vec[-1] += 1
#   return vec

In [92]:
# X = []
# for text in train_df['text']:
#   X.append(sentence_vector(text))
# X = np.array(X)
# X.shape

In [93]:
class LogisticRegression(torch.nn.Module):

  def __init__(self, input_dim, classes) -> None:
    super().__init__()
    self.linear = torch.nn.Linear(input_dim, classes)
    self.sigmoid = torch.nn.Sigmoid()
  
  def forward(self, x):
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

In [94]:
def train(x, y, model, loss, optimizer, ephocs):
  for epoch in epochs:
    y_pred = model(x)
    pred_loss = loss(y, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


In [95]:
from gensim.models import Word2Vec
import gensim

In [96]:
# cleaned_list = []
# i = 0
# for text in train_df['text']:
#   clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
#   tweet = clean_tweet.lower()
#   final = tweet.encode('utf-8').split()
#   cleaned_list.append(final)
# train_df['clean_tweet'] = cleaned_list
# train_df


In [97]:
train_df['text_clean'] = train_df['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
train_df

,id,keyword,location,text,target,text_clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, this, photo, from, ruby, ala..."
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,"[two, giant, cranes, holding, bridge, collapse..."
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,"[aria_ahrary, thetawniest, the, out, of, contr..."
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"[utc, km, of, volcano, hawaii, http, co, zdtoy..."
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,"[police, investigating, after, an, bike, colli..."


In [98]:
embedding_w2v = Word2Vec(train_df['text_clean'], min_count=1,
                              vector_size=300, window=5)
embedding_w2v

In [102]:
embedding_w2v.wv.most_similar('fire')

[('into', 0.9999105334281921),
 ('emergency', 0.9999033808708191),
 ('were', 0.9999018311500549),
 ('for', 0.9999017715454102),
 ('ûªs', 0.999893844127655),
 ('police', 0.9998905658721924),
 ('of', 0.9998878240585327),
 ('from', 0.9998831748962402),
 ('train', 0.9998816847801208),
 ('near', 0.9998810291290283)]

In [109]:
X_train = train_df['text_clean']
words = set(embedding_w2v.wv.index_to_key )
X_train_vect = np.array([np.array([embedding_w2v.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_train_vect

<ipython-input-109-b865ced66570>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([embedding_w2v.wv[i] for i in ls if i in words])


array([array([[-0.06579447,  0.49924138, -0.06560662, ..., -0.26665354,
                0.3875371 , -0.26652   ],
              [-0.00228907,  0.01413311, -0.00147193, ..., -0.0114549 ,
                0.01284711, -0.00638625],
              [-0.11508578,  0.94989294, -0.13690977, ..., -0.5211331 ,
                0.70936584, -0.50401944],
              ...,
              [-0.00249095,  0.01038795, -0.00218319, ..., -0.00614494,
                0.01197165, -0.00738899],
              [-0.07586714,  0.59824103, -0.07863354, ..., -0.31780103,
                0.46438846, -0.3184371 ],
              [-0.09630156,  0.7994274 , -0.11431278, ..., -0.4325103 ,
                0.6129484 , -0.42738736]], dtype=float32)              ,
       array([[-0.04161074,  0.33210063, -0.04291759, ..., -0.17803788,
                0.25943756, -0.17865285],
              [-0.10577738,  0.7977128 , -0.09752104, ..., -0.4242854 ,
                0.62885064, -0.43441552],
              [-0.05364077,  0.3633508

In [112]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
X_train_vect_avg[0].size

300

In [122]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='none').fit(X_train_vect_avg, train_labels)
y_prob= model.predict_proba(X_train_vect_avg)[:, 1]
y_pred = np.round(y_prob)
acc = np.sum(y_pred == train_labels) / len(train_labels)
acc

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.723893340338894